In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder


from keras import Sequential
from keras.layers import Dense
import tensorflow as tf

In [2]:
df = pd.read_csv('Datasets/Churn_Modelling.csv',index_col='RowNumber')
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.duplicated().sum()

0

In [5]:
df.isna().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
df.drop(columns=['CustomerId','Surname'], inplace=True)

In [8]:
cat_columns = df.select_dtypes(include='object').columns
num_columns = df.select_dtypes(exclude='object').columns


df[cat_columns].nunique()

Geography    3
Gender       2
dtype: int64

In [10]:
X = df.drop(columns=['Exited'])
y = df['Exited']

In [17]:
cat_columns = X.select_dtypes(include='object').columns
num_columns = X.select_dtypes(exclude='object').columns

In [18]:
cat_columns.tolist()

[]

In [19]:
X[cat_columns.tolist()] = OrdinalEncoder().fit_transform(X[cat_columns.tolist()])
X[num_columns] = StandardScaler().fit_transform(X[num_columns])

y = LabelEncoder().fit_transform(y)

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

In [22]:
model = Sequential(
    [
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)
model.compile(loss='binary_crossentropy', metrics=['recall'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

class_weight = {
    0: 1,
    1: 4
}

history = model.fit(X_train,y_train, validation_data=(X_val, y_val), epochs=100,callbacks=[early_stop], class_weight=class_weight)


Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0579 - recall: 0.6729 - val_loss: 0.5670 - val_recall: 0.8012
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - loss: 0.9038 - recall: 0.7482 - val_loss: 0.5306 - val_recall: 0.8135
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - loss: 0.8537 - recall: 0.7486 - val_loss: 0.4883 - val_recall: 0.7768
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - loss: 0.7945 - recall: 0.7537 - val_loss: 0.4994 - val_recall: 0.7920
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7812 - recall: 0.7742 - val_loss: 0.4958 - val_recall: 0.8226
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 0.7893 - recall: 0.7843 - val_loss: 0.4886 - val_recall: 0.8104
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - loss: 0.7589 - recall: 0.7785 - val_loss: 0.4755 - val_recall: 0.8043
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7648 - recall: 0.7885 - val_loss: 0.4682 - val_recal

In [23]:
y_pred = model.predict(X_val)

from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(y_val, y_pred>0.5)

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step


array([[996, 277],
       [ 77, 250]])